__Sometimes it can be quite unpredictable to determine whether a customer would be able to repay their loan. The primary factors to determine whether the customer will be able to payback is a challenging and unpredictable process. It can also be time consuming and tedious considering the number of factors involved during the loan application stage from the customer's end. By analyzing the data with over 25 features, we aim to determine factors that will play a key role in creating a predictive model to assist banks in forecasting whether a customer would be able to repay a loan amount or not. The input parameters from customers will be used against the model to determine the probability of payback or the chances of a loan getting charged-off. This can certainly assist the banks in predicting and accordingly making a decision before issuing a particular loan. The predictive model can replace the initial steps of the prequalification process across all financial institutions thus saving time and efforts of both customers and loan representatives.__

__Dataset:__ Data is collected from Lending Club Loan Data website which contains information for all loans issued through 2007-2015 to forecast which customer will repay the loan amount in given time span.

__Goal of the Project:__ To forecast the probability of a customer being able to pay off their loan.

In [2]:
# Imports and setup
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
loans = pd.read_csv('/dbfs/FileStore/tables/loan.csv', low_memory = False)

## Exploring the Dataset

In [5]:
loans.head()

In [6]:
# Let's take a look at the different columns and what data they contain
#cols = loans.columns[0:10]  # cycle through 0:10, 10:20, ...
cols = ['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length']  # or pick specific columns
print(cols)
for col in cols:
    print(loans[col].describe())  # describe one by one in case of mixed types

In [7]:

# Parse term durations: ' 36 months' -> 36 (numeric)
print("term before:-")
print(loans.term.head())
loans.term = pd.to_numeric(loans.term.str[:3])
print("term after:-")
print(loans.term.head())

In [8]:
# Parse emp_length: '< 1 year' -> 1.0, '1 year' -> 1.0, '7 year' -> 7.0, etc. (numeric)
print("emp_length before:-")
print(loans.emp_length.head())
loans.emp_length = loans.emp_length.str.extract("(\d+)", expand=False).map(float)
print("emp_length after:-")
print(loans.emp_length.head())

In [9]:
# For the categorical variables we have replaced NaN with 'Unknown'.
# We will fill verification_status_joint using the value in verification_status as these are all individual applications and these values are not filled out.
loans['verification_status_joint'].fillna(loans['verification_status'], inplace=True)

In [10]:
strColumns = loans.select_dtypes(include=['object']).columns.values
loans[strColumns] = loans[strColumns].fillna('Unknown')

In [11]:
#Check that all the NaN values have been replaced
loans.select_dtypes(exclude=[np.number]).isnull().sum()

In [12]:
# First we will check the number of missing values for each of the columns
loans.select_dtypes(include=[np.number]).isnull().sum()

In [13]:
# The first columns that we are going to update are annual_inc_joint, dti_joint and verification_status_joint. For individual accounts these are blank but we want to use the joint values so we will populate these with the individual values for individual accounts.
loans[loans['application_type'] != 'INDIVIDUAL']['annual_inc_joint'].isnull().sum()

In [14]:
loans['annual_inc_joint'].fillna(loans['annual_inc'], inplace=True)
loans['dti_joint'].fillna(loans['dti'], inplace=True)

In [15]:
#For the remainder of the missing values we are going to fix the missing values by replacing any NaN values with the mean values
strColumns = loans.select_dtypes(include=[np.number]).columns.values
loans[strColumns] = loans[strColumns].fillna(loans[strColumns].mean())

In [16]:
# What is the distribution of loans by status?
loans_by_status = loans.groupby('loan_status')
print(loans_by_status['loan_status'].count())
loans_by_status['loan_status'].count().plot(kind='bar')

In [17]:
# What is the distribution of loans by purpose?
loans_by_purpose = loans.groupby('purpose')
print(loans_by_purpose['purpose'].count())
loans_by_purpose['purpose'].count().plot(kind='bar')
display()

In [18]:
# What is the distribution of loans by term?
loans_by_term = loans.groupby('term')
print(loans_by_term['term'].count())
loans_by_term['term'].count().plot(kind='bar')
display()

In [19]:
# Let's setup a binary classification target 'default': 0 => Fully Paid, 1 => Charged Off
loans_subset = loans.copy()
loans_subset['default'] = None
loans_subset.loc[(loans_subset.loan_status == 'Fully Paid') | (loans_subset.loan_status == 'Does not meet the credit policy. Status:Fully Paid'), 'default'] = 0
loans_subset.loc[(loans_subset.loan_status == 'Charged Off') | (loans_subset.loan_status == 'Does not meet the credit policy. Status:Charged Off'), 'default'] = 1

# Drop loans that haven't been terminated yet (we don't know what their final status will be)
loans_subset = loans_subset[~loans_subset.default.isnull()]
print("Data subset size: {}".format(loans_subset.shape))

# Re-encode 'default' column as numeric (0 or 1)
loans_subset['default'] = pd.to_numeric(loans_subset['default'])

## Detailed Information of the features

## Input Variables:


## Application Stage Data:

__member_id:__ The month which the loan was funded

__issue_d:__ A unique LC assigned Id for the borrower member.



## Loan Application Details:

__addr_state:__	The state provided by the borrower in the loan application

__emp_length:__	Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. 

__emp_title:__	The job title supplied by the Borrower when applying for the loan.

__home_ownership:__	The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.

__installment:__	The monthly payment owed by the borrower if the loan originates.

__int_rate:__	Interest Rate on the loan

__loan_amnt:__	The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

__purpose:__	A category provided by the borrower for the loan request. 

__term:__	The number of payments on the loan. Values are in months and can be either 36 or 60.

__title:__	The loan title provided by the borrower

__verified_status_joint:__	Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified

__zip_code:__	The first 3 numbers of the zip code provided by the borrower in the loan application.



## Borrowers Creditworthiness:

__annual_inc:__	The self-reported annual income provided by the borrower during registration.

__delinq_2yrs:__	The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years

__dti:__	A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.

__earliest_cr_line:__	The month the borrower's earliest reported credit line was opened

__inq_last_6mths:__	The number of inquiries in past 6 months (excluding auto and mortgage inquiries)

__mths_since_last_delinq:__	The number of months since the borrower's last delinquency.

__open_acc:__	The number of open credit lines in the borrower's credit file.

__revol_bal:__	Total credit revolving balance

__revol_util:__	Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

__total_acc:__	The total number of credit lines currently in the borrower's credit file



## Public Records:

__mths_since_last_record:__	The number of months since the last public record.

__pub_rec:__	Number of derogatory public records



## Output Variable (Desired Target)

__default:__ has the customer able to pay off their loan? (binary: 'yes','no')

In [22]:
# We only want to keep information that is available at loan *application* stage
application_cols = [
    # Identifiers and dates
    #'id',  # used as index column
    'member_id',
    'issue_d',
    
    # Loan application details
    #'application_type',  # all 'INDIVIDUAL'
    'loan_amnt',  # $ applied for
    'term',  # 36 or 60 months
    'int_rate',  # % annual (?) interest rate
    'installment',  # $ monthly payment
    'emp_title',  # employee/employer title
    'emp_length',  # 0-10+ years
    'home_ownership',  # RENT, OWN, MORTGAGE, etc.
    'verification_status',  # mostly 'Not Verified'
    #'verification_status_joint',  # all 0
    'purpose',  # 'debt_consolidation', 'small_business', etc.
    'title',  # text
    #'desc',  # text, too verbose, may contain updates after application stage
    'zip_code',  # 100XX
    'addr_state',  # covered by zip_code?
    
    # Additional loan listing details
    #'initial_list_status',  # all 'f'
    #'policy_code',  # all 1
    #'url',  # unqiue per loan

    # Borrower's creditworthiness
    'annual_inc', #'annual_inc_joint',  # income ($; individual only, no joint loans)
    'dti', #'dti_joint',  # debt-to-income ratio (%; individual only, no joint loans)
    'revol_bal', 'revol_util',  # revolving accounts: balance ($), utilization (%)
    #'tot_cur_bal', 'max_bal_bc',  # overall balance: total current, max; all null
    'earliest_cr_line', 'total_acc', 'open_acc',  # credit accounts
    'inq_last_6mths', #'inq_last_12m', 'inq_fi',  # credit inquiries (only 6 mths available)
    'delinq_2yrs', 'mths_since_last_delinq', #'acc_now_delinq',  # delinquency (acc_now_delinq is mostly 0)
    #'tot_coll_amt', 'collections_12_mths_ex_med',  # collections; all null or 0
    #'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',  # installment accounts; all null
    #'open_acc_6m', 'open_rv_12m', 'open_rv_24m', 'total_rev_hi_lim', 'total_cu_tl', 'all_util', # revolving trading accounts; all null
    
    # Public records
    'pub_rec', 'mths_since_last_record',
    #'mths_since_last_major_derog',  # all null

    # Loan rating as determined by lender (potential multi-class targets to predict?)
    #'grade',
    #'sub_grade',

    # Desired binary target to predict
    'default'
]

loans_small = loans_subset[application_cols]

# Check selected data subset
print("Small dataset has {} rows, {} columns:".format(len(loans_small), len(loans_small.columns)))
print(loans_small.head())
print("Class distribution:")
print(loans_small.groupby('default')['default'].count())

In [23]:
# Displaying Heatmap of Correlation Matrix

corr=loans_small.corr()
corr = (corr)
plt.figure(figsize=(40,40))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws=

{'size': 15},
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
sns.plt.title('Heatmap of Correlation Matrix')
display()

In [24]:
loans_small1 = loans_small

In [25]:
loans_small.head(5)

### Feature Extraction

In [27]:
# removing irrelevant columns after analyzing heatmap

loansdf = loans_small1.drop([
'member_id',
'issue_d',
'emp_title',
'title',
'zip_code',
'addr_state',
'earliest_cr_line',
'total_acc',
'open_acc',
'inq_last_6mths',
'delinq_2yrs',
'mths_since_last_delinq',
'pub_rec',
'mths_since_last_record',
'default' 
],axis=1)

In [28]:
# Displaying final heatmap of correlation matrix

corr=loansdf.corr()
corr = (corr)
plt.figure(figsize=(15,15))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws=

{'size': 20},
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
sns.plt.title('Heatmap of Correlation Matrix')
display()

### Extracting the desired features and target column into a Data Frame

In [30]:
# Specify a subset of feature columns and a target to predict ('default')
feature_cols = [
    'loan_amnt', 'term', 'int_rate', 'installment', 'purpose',
    #'emp_title', # free text
    'emp_length', 'home_ownership',
    #'zip_code', 'addr_state',  # categorical, but too many levels
    'annual_inc', 'dti',
    'revol_bal', 'revol_util',
    'verification_status'
]

target_col = 'default'

# Create the final dataset we'll use for classification
keep_cols = feature_cols + [target_col]
loans_final = loans_small[keep_cols]

# Drop samples with null values (few enough that we can ignore)
loans_final.dropna(inplace=True)

print("Final dataset: {} features, {} samples".format(len(loans_final.columns), len(loans_final)))
print(loans_final.head())
print("Final class distribution (after dropping nulls):")
class_counts = loans_final.groupby(target_col)[target_col].agg({
    'count': len,
    'ratio': lambda x: float(len(x)) / len(loans_final)
})
print(class_counts)

# Extract desired features and target column
X = loans_final[feature_cols]
y = loans_final[target_col]
print("{} features: {}".format(len(X.columns), X.columns))
print("Target: {}".format(y.name))

In [31]:
# label encoding the categorical values for easier processing
from sklearn.preprocessing import LabelEncoder
testDF = loans_final.apply(LabelEncoder().fit_transform)

In [32]:
print(testDF)

In [33]:
# normalizing the values for better results
from sklearn import preprocessing

normalized_npArray = preprocessing.normalize(testDF)
normalized_df = pd.DataFrame(normalized_npArray, columns = testDF.columns)
normalized_df.default = testDF.default

### We can find the correlation of feature among themselves by performing the correlation on the dataframe

In [35]:
# finding correlation of target with other variables
corr = abs(normalized_df.corr())
corr.sort_values(["default"], ascending = False, inplace = True)

corr_df = pd.DataFrame(corr.default)
corr_df['Features'] = corr_df.index
corr_df = corr_df[['Features', 'default']]
corr_df = corr_df[corr_df.default != 1]
sns.barplot(x = 'Features', y = 'default', data = corr_df)
plt.xticks(rotation=60)
plt.show()
display()

In [36]:
# Encode categorical variables among features
categorical_vars = ['home_ownership', 'purpose', 'verification_status']
X = pd.get_dummies(X, columns=categorical_vars, drop_first=True)
print("{} features after encoding categorical variables: {}".format(len(X.columns), X.columns))

In [37]:
original_df = testDF.copy()
# Always scale your data with a mean of 0 and a standard deviation of 1.. the models can work better in that regards
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(testDF)
# test = StandardScaler().fit(X).transform(X)
X = scaler.transform(testDF)
X = pd.DataFrame(X , columns=original_df.columns)
X.head()

In [38]:
# Split into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print("Training set: {} samples, test set: {} samples".format(len(X_train), len(X_test)))
print("Training set: {} samples, test set: {} samples".format(len(y_train), len(y_test)))

In [39]:
# Common sklearn imports
from sklearn.metrics import classification_report

# Define a simple train-predict utility function
def train_predict(clf, X_train, X_test, y_train, y_test):
    """Train clf on <X_train, y_train>, predict <X_test, y_test>; return y_pred."""
    print("Training a {}...".format(clf.__class__.__name__))
    %time clf.fit(X_train, y_train)
    print(clf)
    
    print("Predicting test labels...")
    y_pred = clf.predict(X_test)
    return y_pred

In [40]:
loans_final.head(10)

__In order to perform the analysis on data, we divided it into training, validation and testing data. The ratio we took is 60:30:10 respectively.__

In [42]:
#In order to avoid biasness, we will divide the dataset by splitting into yes and no and then taking equal proportions from both
# This is done to ensure that that the percentage of yes and no in tarining, validationa and testing remain constant

yes_DF = loans_final[loans_final['default']==1]
no_DF = loans_final[loans_final['default']==0]

#creating spark df from pandas df
test_sdf = sqlContext.createDataFrame(loans_final)
yes_sdf = sqlContext.createDataFrame(yes_DF)
no_sdf = sqlContext.createDataFrame(no_DF)

#dividing data into training, validation and testing
training_yes, validation_yes, testing_yes = yes_sdf.randomSplit([0.6, 0.3, 0.1], seed=0)
training_no, validation_no, testing_no = no_sdf.randomSplit([0.6, 0.3, 0.1], seed=0)

#Combining the yes/no dataframes into single dataframes
training = training_yes.unionAll(training_no)
validation = validation_yes.unionAll(validation_no)
testing = testing_yes.unionAll(testing_no)

#show the count
print(training.count())
print(validation.count())
print(testing.count())

In [43]:
from pyspark.sql import functions as fn
from pyspark.ml import Pipeline, PipelineModel, feature, regression, classification, evaluation

va = feature.VectorAssembler(inputCols=['loan_amnt', 'term', 'int_rate', 'installment', 
    #'emp_title', # free text
    'emp_length', 
    #'zip_code', 'addr_state',  # categorical, but too many levels
    'annual_inc', 'dti',
    'revol_bal', 'revol_util'], outputCol='features')

__ We plan to use various classification methods to find the best model among the different models we would try to implement.__

In [45]:
#Logistic Regression
from pyspark.ml.classification import LogisticRegression
lr = classification.LogisticRegression(labelCol="default" , featuresCol="features")
lr_pipeline = Pipeline(stages=[va, lr])
lr_pipeline_model = lr_pipeline.fit(training)

In [46]:
#Evaluating the accuracy of logistic model
from pyspark.ml.evaluation import BinaryClassificationEvaluator
bce = BinaryClassificationEvaluator()
bce.setLabelCol('default')
bce.evaluate(lr_pipeline_model.transform(validation))

__The accuracy of Logistic Regression is around 69.41%.__

In [48]:
#Trying to increase the complexity of the models by polynomial expansion

polynomial_levels = [2,3,4]

for level in polynomial_levels:
    pe = feature.PolynomialExpansion(degree=level, inputCol='features', outputCol='all_features')
    lr_2 = classification.LogisticRegression(labelCol="default" , featuresCol="all_features")
    pipeline_2 = Pipeline(stages=[va, pe,lr_2])
    lr_pipeline_model_2 = pipeline_2.fit(training)
    print('Accuracy with', level , 'degrees:', bce.evaluate(lr_pipeline_model_2.transform(validation)))

__After adding more polynomial levels(2,3,4), it does not show a significance increase in the accuracy level.__

In [50]:
#Adding elastic net regularization to see if the accuracy can be increased over the models.

polynomial_levels = [1,2,3,4]

reg_param = 0.02
en_param = 0.3

for level in polynomial_levels:
    pe = feature.PolynomialExpansion(degree=level, inputCol='features', outputCol='all_features')
    lr_2 = classification.LogisticRegression(labelCol="default" , featuresCol="all_features").setRegParam(reg_param).\
    setElasticNetParam(en_param)
    pipeline_2 = Pipeline(stages=[va, pe,lr_2])
    lr_pipeline_model_2 = pipeline_2.fit(training)
    print('Accuracy with', level , 'degrees:', bce.evaluate(lr_pipeline_model_2.transform(validation)))

__After adding the regularization parameters, it did not have a major change in the accuracy.__

In [52]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
li = StringIndexer(inputCol="default", outputCol="indexedLabel").fit(test_sdf)

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="features")

# Convert indexed labels back to original labels.
lc = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=li.labels)

# Chain indexers and forest in a Pipeline
rf_pipeline = Pipeline(stages=[li, va, rf, lc])

# Train model.  This also runs the indexers.
rf_model = rf_pipeline.fit(training)

# Compute accuracy
evaluator = MulticlassClassificationEvaluator(\
                                              labelCol="indexedLabel",\
                                              predictionCol="prediction",\
                                              metricName="accuracy")

print(evaluator.evaluate(rf_model.transform(validation)))

__After performing Random Forest, it has proved to be a better model than logistic regression with over 80% accuracy over validation.__

__Now we aim to implement a better random forest model by pruning the various number of trees.__

In [55]:
#Building a grid of number of trees using ParamGridBuilder
from pyspark.ml.tuning import ParamGridBuilder

grid = ParamGridBuilder().\
    addGrid(rf.numTrees, range(5,20)).\
    build()

In [56]:
 #Fitting the training data on 
all_models = []
i = 5

evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

for j in range(len(grid)):
    model = rf_pipeline.fit(training, grid[j])
    accuracy = evaluator.evaluate(model.transform(validation))
    print("Accuracy with " + str(i) + " trees: " + str(accuracy))
    i += 1

__Random Forest model with 7 trees gave us the maximum accuracy. Thus, we plan to use it as the best model.__

In [58]:
# Using Random Forest with 7 trees to find accuracy on the test dataset
rf7 = RandomForestClassifier(labelCol="indexedLabel", featuresCol="features", numTrees=7)

#Chain indexers and forest in a Pipeline
rf_pipeline = Pipeline(stages=[li, va, rf7, lc])

#Train model.  This also runs the indexers.
rf_model = rf_pipeline.fit(training)

#Compute accuracy
print(evaluator.evaluate(rf_model.transform(testing)))

__The accuracy of Random Forest is 81.86%__

In [60]:
#Finding the importance of the features used
importances = rf_model.stages[2].featureImportances
importances

__It is evident from the sparse vector that some of the features are more important than the other in the model we have built. Hence, the banks can focus on those features more than the other features.__

In [62]:
predictions = lr_pipeline_model.transform(testing)

from pyspark.mllib.evaluation import BinaryClassificationMetrics as metric
from sklearn.metrics import roc_curve, auc, classification_report

results = predictions.select(['default', 'prediction'])
 
## prepare score-label set
results_collect = results.collect()
results_list = [(float(i[0]), float(i[1])) for i in results_collect]
 
fpr = dict()
tpr = dict()
roc_auc = dict()
 
y_test1 = [i[1] for i in results_list]
y_score1 = [i[0] for i in results_list]
 
fpr, tpr, threshold = roc_curve(y_test1, y_score1)
roc_auc = auc(fpr, tpr)
 
%matplotlib inline
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

display()

__We created the ROC curve by plotting True Positive Rate (TPR) against the False Positive Rate (FPR) at various thresholds. The Area under the curve (AUC) that we have plotted is on the higher side, which proves that the accuracy of our model is good.__

In [64]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test1, y_score1)
conf_mat

## if mlxtend package is installed, the following code will generate a colored confusion matrix
#from mlxtend.plotting import plot_confusion_matrix
#fig, ax = plot_confusion_matrix(conf_mat=conf_mat, hide_ticks=True, cmap='flag')
#plt.show()

In [65]:
import matplotlib.pyplot as plt

plt.matshow(conf_mat)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

display()

__Here, the confusion matrix depicts the number of true positives and false negatives which is large. It means that the model is giving out good results.__

In [67]:
print (classification_report(y_test1, y_score1))

__The classification report shows that our model has given results with a high precision, that is desired by the bank to focus if a customer would be able to repay their loan or not. Moreover, this would assist them in giving loan to customers again, if they are able to repay their loan. Hence, the bank can use this model to forecast if the customer would be able to repay the loan amount in the given time span.__

### Appendix

#### Version of different packages used in the project:

__Python 2.7.6__

__sklearn 0.17.1__

__matplotlib 1.5.3__

__numpy 1.11.1__

__pandas 0.18.1 __

__seaborn 0.7.1__